In [1]:
# -------------------------------------------------------------
# Clone the repo
# -------------------------------------------------------------
import shutil, os, subprocess, sys, gc, time, threading, psutil

if os.path.isdir("/kaggle/working/Chatterbox-TTS-Server"):
    shutil.rmtree("/kaggle/working/Chatterbox-TTS-Server")
!git clone https://github.com/devnen/Chatterbox-TTS-Server.git /kaggle/working/Chatterbox-TTS-Server

# Core ML packages
!pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 \
    --index-url https://download.pytorch.org/whl/cu121 -q
!pip install git+https://github.com/devnen/chatterbox.git -q

# Server‑side requirements
!pip install fastapi uvicorn pyyaml soundfile librosa safetensors \
    python-multipart requests jinja2 watchdog aiofiles unidecode inflect tqdm \
    pydub audiotsm -q

# Optional – may fail on Kaggle, ignore if it does
try:
    !pip install parselmouth -q
except Exception:
    print("⚠️  `parselmouth` could not be installed – skipping.")

Cloning into '/kaggle/working/Chatterbox-TTS-Server'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 302 (delta 138), reused 119 (delta 119), pack-reused 139 (from 1)
Receiving objects: 100% (302/302), 18.94 MiB | 34.09 MiB/s, done.
Resolving deltas: 100% (147/147), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB

In [2]:
# -------------------------------------------------------------
# Change into the repo folder
# -------------------------------------------------------------
os.chdir("/kaggle/working/Chatterbox-TTS-Server")
print("📁 Working directory:", os.getcwd())

📁 Working directory: /kaggle/working/Chatterbox-TTS-Server


In [3]:
# -------------------------------------------------------------
# 4️⃣  Launch the TTS server with memory‑friendly flags
# -------------------------------------------------------------
import gc, time, threading, uvicorn, psutil, torch

# ---- GPU‑optimisation flags (پہلے چیک کر لیں)
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

port = 8004

# ---- Kill any stray process that already uses the port -----------------
for conn in psutil.net_connections(kind='tcp'):
    if conn.laddr and conn.laddr.port == port:
        pid = conn.pid
        if pid is None: continue
        try:
            p = psutil.Process(pid)
            print(f"⚠️  Killing stray process {p.name()} (pid={pid}) that uses port {port}")
            p.kill()
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass

# ---- Garbage‑collect before loading the heavy model --------------------
gc.collect()

def run_server():
    try:
        from server import app          # <-- provided by the repo
        print("✅ Server is starting …")
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=port,
            log_level="warning",
            access_log=False
        )
    except Exception as e:
        print(f"❌ Server crashed: {e}")

# ---- Run in a daemon thread -------------------------------------------
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("⏳ Waiting for the model to load …")
time.sleep(30)          # GPU پر وزن لوڈ ہو کر بنڈر تقریباً 5‑10 سیکنڈ لگتے ہیں
print("🚀 Server should be up now (verify with the ngrok tunnel).")


⏳ Waiting for the model to load …


2025-08-15 12:53:09.291887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755262389.534510     126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755262389.601531     126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

🚀 Server should be up now (verify with the ngrok tunnel).


In [4]:
# -------------------------------------------------
# 0️⃣ npm (node‑package‑manager) انسٹال (اگر پہلے سے نہیں)
# -------------------------------------------------
# Kaggle کے اکثر رن‑ٹائم میں npm پہلے سے موجود ہوتا ہے
!npm --version   # اگر یہ کام نہ کرے تو ایڈمن موڈ سے apt‑get install npm کر لیں
#   (عام طور پر نیچے کی کمانڈ چلانے سے کافی ہے)


10.8.2


In [5]:
!npm install -g localtunnel   # ایک بار کا کام، مزید بار نہیں چلانا پڑے گا


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 2s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼npm notice
npm notice New major version of npm available! 10.8.2 -> 11.5.2
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.5.2
npm notice To update run: npm install -g npm@11.5.2
npm notice
⠼

In [6]:
import subprocess, re, sys, atexit, time, requests

PORT = 8004                                 # FastAPI جس پورٹ پر چل رہا ہے

# -------------------------------------------------
# 1️⃣  localtunnel کو بیک‑گراؤنڈ میں سٹارٹ
# -------------------------------------------------
lt_process = subprocess.Popen(
    ["lt", "--port", str(PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# -------------------------------------------------
# 2️⃣  ٹنل کی URL پڑھیں
# -------------------------------------------------
public_url = None
while True:
    line = lt_process.stdout.readline()
    if not line:               # پراسس ختم ہو گیا
        break
    # مثال: "your publicly accessible URL is: https://splash‑bright‑tree.loca.lt"
    m = re.search(r"https?://[^\s]+", line)
    if m:
        public_url = m.group(0)
        break

if public_url is None:
    print("❌  LocalTunnel سے URL نہیں ملی – لاگز چیک کریں")
    sys.exit(1)

# -------------------------------------------------
# 3️⃣  ٹنل پاس ورڈ (سرور کا پبلک IP) حاصل کریں
#    • یہ ایند پوائنٹ ہمیشہ ایک سادہ ٹیکسٹ ریٹرن کرتا ہے
# -------------------------------------------------
try:
    pw_resp = requests.get("https://loca.lt/mytunnelpassword")
    tunnel_password = pw_resp.text.strip()
except Exception as e:
    tunnel_password = "❓  پاس ورڈ حاصل نہیں ہو سکا"
    print("⚠️  tunnel‑password fetch error:", e)

# -------------------------------------------------
# 4️⃣  نتیجہ پرنٹ کریں
# -------------------------------------------------
print("\n🚀  Public URL (localtunnel – ٹوکن کی ضرورت نہیں)")
print(public_url)

print("\n🔐  Tunnel password (یہ پبلک IP ہے، اس کو براؤزر میں داخل کریں)")
print(tunnel_password)

print("\n📖  Swagger UI   →", f"{public_url}/docs")
print("🗝️  اگر براوزر میں پہلی بار کھولیں تو پاس ورڈ مانگے گا – اوپر والا پاس ورڈ یہی داخل کریں۔\n")

# -------------------------------------------------
# 5️⃣  نوٹ‑بُک ختم ہونے پر lt پراسس بند کریں
# -------------------------------------------------
atexit.register(lambda: lt_process.terminate())



🚀  Public URL (localtunnel – ٹوکن کی ضرورت نہیں)
https://tough-comics-march.loca.lt

🔐  Tunnel password (یہ پبلک IP ہے، اس کو براؤزر میں داخل کریں)
34.32.195.16

📖  Swagger UI   → https://tough-comics-march.loca.lt/docs
🗝️  اگر براوزر میں پہلی بار کھولیں تو پاس ورڈ مانگے گا – اوپر والا پاس ورڈ یہی داخل کریں۔



<function __main__.<lambda>()>

 Kaggle Notebook - AI Tool
To use this notebook, please follow these steps:
 * Click on the provided link below to open the  Kaggle file.
 * In the code cell, add your personal ngrok authentication token.
 * If you don't have a token, you can get one for free from the official ngrok website:
Ngrok Website: [https://dashboard.ngrok.com/get-started/your-authtoken](http://)

 👇🏼👇🏼👇🏼👇🏼

In [7]:
!pip install pyngrok -q

from pyngrok import ngrok

NGROK_AUTH_TOKEN = "👉 Your TOKEN"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(port, "http")
print("\n🔗 Ngrok tunnel URL:", public_url)
print("Open the docs UI 👉", f"{public_url}/docs")

✅ Server is starting …


ve.safetensors:   0%|          | 0.00/5.70M [00:00<?, ?B/s]

t3_cfg.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

ERROR:  authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.
ERROR:  Your authtoken: 👉 Your TOKEN
ERROR:  Instructions to install your authtoken are on your ngrok dashboard:
ERROR:  https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_105
ERROR:  https://ngrok.com/docs/errors/err_ngrok_105
ERROR:  


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: 👉 Your TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.